In [14]:
import pandas as pd
import os
import glob
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Rescaling

Import all data as one dataframe

In [2]:
data_dir = 'trainingdata_new'
all_data = glob.glob(os.path.join(data_dir, "*.csv"))
combined_data = pd.concat([pd.read_csv(file) for file in all_data], ignore_index=True)
combined_data.drop(["Unnamed: 0"], axis = 1)

,Wavelength,V1,V2,Gain,Out
0,495,0.050000,0.050000,4,0.072094
1,495,0.130612,0.050000,4,0.070441
2,495,0.211224,0.050000,4,0.072832
3,495,0.291837,0.050000,4,0.073059
4,495,0.372449,0.050000,4,0.073020
...,...,...,...,...,...
1395,590,3.677551,0.372449,4,0.061949
1396,590,3.758163,0.291837,4,0.062184
1397,590,3.838776,0.211224,4,0.063187
1398,590,3.919388,0.130612,4,0.063203


Adjusts output voltage for the gain factor

In [3]:
combined_data.loc[0,'Out']
new_out = []
for i in range(len(combined_data)):
    adj_out = combined_data.loc[i,'Out']*combined_data.loc[i,'Gain']
    new_out.append(adj_out)
combined_data['Out'] = new_out

Fitting model as shown in svm_regression_test.ipynb (will go into further detail)

In [68]:
x = combined_data['V1'].to_numpy()
y = combined_data['V2'].to_numpy()
z = combined_data['Gain Volts'].to_numpy(dtype = 'object')

X = np.column_stack((x, y))

param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1]}
grid_search = GridSearchCV(SVR(kernel='rbf'), param_grid, cv=5)
grid_search.fit(X, z)
best_c = grid_search.best_params_['C']
best_gamma = grid_search.best_params_['gamma']

model = SVR(kernel='rbf', C=best_c, gamma=best_gamma) 
model.fit(X,z)
x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1 
xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max, 500),
                       np.linspace(x2_min, x2_max, 500))
xx_input = np.column_stack((xx1.ravel(), xx2.ravel()))
y_pred = model.predict(xx_input)

# Attempting Neural Network Fitting
The above fit is a bit rough, and part of the problem is that it's wavelength agnostic. Of course, we do have a wavelength dependency in our data. However, the reason it's wavelength agnostic is because (from what I can gather) getting the predictions we'd like from a 4D SVM regressor is difficult. The current goal is to give two inputs (wavelength and desired polarization angle) and receive the input voltages needed for the LCVR's to produce the angle. Therefore a neural net may be more apt to fit this data.

In [5]:
## Test neural network code created with Gemini (modified slightly by me)

import tensorflow as tf
from tensorflow.keras import layers

# Converts dataframe to tensor
# Note x is "input" wavelength, output voltage, Y is LCVR Volts
x = combined_data.iloc[:, [1, 5]]
y = combined_data.iloc[:, [2, 3]] 
X = tf.convert_to_tensor(x)
Y = tf.convert_to_tensor(y)

# Split into training and validation sets
# Splitting ratio (e.g., 80% for training)
split_ratio = 0.8

# Calculate the split index
split_index = int(split_ratio * len(X))

# Splitting the tensors
X_train = X[:split_index]
X_val = X[split_index:]
Y_train = Y[:split_index]
Y_val = Y[split_index:]

# Model Definition
model = tf.keras.Sequential([
  layers.Dense(10, activation='relu', input_shape=(2,)),  # Hidden layer
  layers.Dense(2)  # Output layer with 2 neurons
])

# Compilation
model.compile(optimizer='adam', loss='mean_squared_error')

# Training
model.fit(X_train, Y_train, epochs=1000, validation_data=(X_val, Y_val), verbose = 2)


Epoch 1/1000
35/35 - 1s - 19ms/step - loss: 51390.3047 - val_loss: 50837.7344
Epoch 2/1000
35/35 - 0s - 2ms/step - loss: 35520.3359 - val_loss: 34679.1953
Epoch 3/1000
35/35 - 0s - 2ms/step - loss: 23923.7480 - val_loss: 22983.6738
Epoch 4/1000
35/35 - 0s - 2ms/step - loss: 15611.9824 - val_loss: 14710.2773
Epoch 5/1000
35/35 - 0s - 2ms/step - loss: 9810.1641 - val_loss: 9034.0352
Epoch 6/1000
35/35 - 0s - 2ms/step - loss: 5912.1157 - val_loss: 5313.9912
Epoch 7/1000
35/35 - 0s - 2ms/step - loss: 3417.4961 - val_loss: 3017.4241
Epoch 8/1000
35/35 - 0s - 2ms/step - loss: 1911.9531 - val_loss: 1664.4960
Epoch 9/1000
35/35 - 0s - 2ms/step - loss: 1044.7532 - val_loss: 904.4114
Epoch 10/1000
35/35 - 0s - 2ms/step - loss: 563.7111 - val_loss: 487.1378
Epoch 11/1000
35/35 - 0s - 2ms/step - loss: 301.7645 - val_loss: 260.5458
Epoch 12/1000
35/35 - 0s - 2ms/step - loss: 160.1362 - val_loss: 138.0415
Epoch 13/1000
35/35 - 0s - 2ms/step - loss: 83.9158 - val_loss: 72.3838
Epoch 14/1000
35/35 - 0

In [7]:
# Prediction
new_input = tf.constant([[500,-.1]])  # Input with 2 known features
prediction = model.predict(new_input)
print(prediction) 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[[2.6466823  0.46177444]]


This is the first iteration. It's a bit better than SVM I believe but it could use some work. Now going to try tuning the model and such to better fit the data

First scaling the data. Going to try the scaling layer included in Tensorflow.keras. Will scale based on a range from 0 to 1 for both which means we need the max values for each of our inputs.

In [16]:
max_wavelength = x['Wavelength'].max()
min_output = x['Out'].min()
max_output = x['Out'].max()
output_range = max_output - min_output

model = tf.keras.Sequential([
  Rescaling(scale=[1./max_wavelength, 1./output_range], offset=[0, min_output]),
  layers.Dense(10, activation='relu', input_shape=(2,)),  # Hidden layer
  layers.Dense(2)  # Output layer with 2 neurons
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, Y_train, epochs=300, validation_data=(X_val, Y_val), verbose = 2)

Epoch 1/300


/home/jimmy/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


35/35 - 1s - 24ms/step - loss: 3.4962 - val_loss: 3.9493
Epoch 2/300
35/35 - 0s - 3ms/step - loss: 3.0213 - val_loss: 3.3918
Epoch 3/300
35/35 - 0s - 2ms/step - loss: 2.6204 - val_loss: 2.9171
Epoch 4/300
35/35 - 0s - 2ms/step - loss: 2.3032 - val_loss: 2.5305
Epoch 5/300
35/35 - 0s - 2ms/step - loss: 2.0682 - val_loss: 2.2563
Epoch 6/300
35/35 - 0s - 2ms/step - loss: 1.9164 - val_loss: 2.0739
Epoch 7/300
35/35 - 0s - 2ms/step - loss: 1.8345 - val_loss: 1.9544
Epoch 8/300
35/35 - 0s - 2ms/step - loss: 1.7911 - val_loss: 1.8980
Epoch 9/300
35/35 - 0s - 2ms/step - loss: 1.7724 - val_loss: 1.8607
Epoch 10/300
35/35 - 0s - 2ms/step - loss: 1.7624 - val_loss: 1.8389
Epoch 11/300
35/35 - 0s - 2ms/step - loss: 1.7555 - val_loss: 1.8310
Epoch 12/300
35/35 - 0s - 2ms/step - loss: 1.7501 - val_loss: 1.8177
Epoch 13/300
35/35 - 0s - 2ms/step - loss: 1.7445 - val_loss: 1.8091
Epoch 14/300
35/35 - 0s - 3ms/step - loss: 1.7389 - val_loss: 1.8018
Epoch 15/300
35/35 - 0s - 3ms/step - loss: 1.7338 - va

This is somewhat helpful, as we have lower RMS error in less time. Now will try adding ReLU layers

In [22]:
max_wavelength = x['Wavelength'].max()
min_output = x['Out'].min()
max_output = x['Out'].max()
output_range = max_output - min_output

model = tf.keras.Sequential([
  Rescaling(scale=[1./max_wavelength, 1./output_range], offset=[0, min_output]),
  layers.Dense(10, activation='relu', input_shape=(2,)),
  layers.Dense(10, activation='relu', input_shape=(2,)),  # Hidden layer
  layers.Dense(2)  # Output layer with 2 neurons
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, Y_train, epochs=300, validation_data=(X_val, Y_val), verbose = 2)

Epoch 1/300
35/35 - 1s - 27ms/step - loss: 3.8983 - val_loss: 4.5996
Epoch 2/300
35/35 - 0s - 2ms/step - loss: 3.6428 - val_loss: 4.3448
Epoch 3/300
35/35 - 0s - 2ms/step - loss: 3.4032 - val_loss: 4.0079
Epoch 4/300
35/35 - 0s - 2ms/step - loss: 3.0920 - val_loss: 3.5524
Epoch 5/300
35/35 - 0s - 2ms/step - loss: 2.7010 - val_loss: 3.0128
Epoch 6/300
35/35 - 0s - 2ms/step - loss: 2.2983 - val_loss: 2.4757
Epoch 7/300
35/35 - 0s - 2ms/step - loss: 1.9639 - val_loss: 2.0556
Epoch 8/300
35/35 - 0s - 2ms/step - loss: 1.7760 - val_loss: 1.8476
Epoch 9/300
35/35 - 0s - 3ms/step - loss: 1.7307 - val_loss: 1.7846
Epoch 10/300
35/35 - 0s - 2ms/step - loss: 1.7247 - val_loss: 1.7780
Epoch 11/300
35/35 - 0s - 2ms/step - loss: 1.7223 - val_loss: 1.7763
Epoch 12/300
35/35 - 0s - 2ms/step - loss: 1.7201 - val_loss: 1.7689
Epoch 13/300
35/35 - 0s - 2ms/step - loss: 1.7175 - val_loss: 1.7754
Epoch 14/300
35/35 - 0s - 2ms/step - loss: 1.7150 - val_loss: 1.7651
Epoch 15/300
35/35 - 0s - 2ms/step - loss:

KeyboardInterrupt: 

This is even better and continually decreases the loss... Sometimes? I need to figure out how to make that more consistent

Rescaling input, target, and validation data. There should be an easier way but I wanted to have the scale factors stored as well for recovery later. (Maybe make this a function when fully implemented?)

NOTE: does not currently store scale factors actually but it does scale properly for now

In [40]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

#Y training
ytrainscale1 = scaler.fit(Y_train[:,0:1])
ytrainscale2 = scaler.fit(Y_train[:,1:])
rs_Y_train = tf.concat([
    tf.convert_to_tensor(scaler.fit(Y_train[:,0:1]).transform(Y_train[:, 0:1])),
    tf.convert_to_tensor(scaler.fit(Y_train[:,1:]).transform(Y_train[:, 1:]))
], axis=1)

# X training
xtrainscale1 = scaler.fit(X_train[:,0:1])
xtrainscale2 = scaler.fit(X_train[:,1:])
rs_X_train = tf.concat([
    tf.convert_to_tensor(scaler.fit(X_train[:,0:1]).transform(X_train[:, 0:1])),
    tf.convert_to_tensor(scaler.fit(X_train[:,1:]).transform(X_train[:, 1:]))
], axis=1)

#Y validation
# Want same scaler on training and validation data
rs_Y_val = tf.concat([
    tf.convert_to_tensor(scaler.fit(Y_train[:,0:1]).transform(Y_val[:, 0:1])),
    tf.convert_to_tensor(scaler.fit(Y_train[:,1:]).transform(Y_val[:, 1:]))
], axis=1)

#X validation
rs_X_val = tf.concat([
    tf.convert_to_tensor(scaler.fit(X_train[:,0:1]).transform(X_val[:, 0:1])),
    tf.convert_to_tensor(scaler.fit(X_train[:,1:]).transform(X_val[:, 1:]))
], axis=1)
print(rs_X_val)

tf.Tensor(
[[0.28       0.7347342 ]
 [0.28       0.7343604 ]
 [0.28       0.7347609 ]
 [0.28       0.74853817]
 [0.28       0.73654981]
 [0.28       0.7347609 ]
 [0.28       0.7343604 ]
 [0.28       0.7344138 ]
 [0.28       0.7342536 ]
 [0.28       0.73673671]
 [0.28       0.7342803 ]
 [0.28       0.7346274 ]
 [0.28       0.7343871 ]
 [0.28       0.7353483 ]
 [0.28       0.74490695]
 [0.28       0.7347609 ]
 [0.28       0.72157104]
 [0.28       0.73182389]
 [0.28       0.71764612]
 [0.28       0.71003658]
 [0.28       0.7351881 ]
 [0.28       0.7337196 ]
 [0.28       0.77053908]
 [0.28       0.79521   ]
 [0.28       0.82957306]
 [0.28       0.8562999 ]
 [0.28       0.86831495]
 [0.28       0.88195872]
 [0.28       0.89063626]
 [0.28       0.8969909 ]
 [0.28       0.90775105]
 [0.28       0.90529464]
 [0.28       0.89990121]
 [0.28       0.8972846 ]
 [0.28       0.89277227]
 [0.28       0.88980856]
 [0.28       0.8781406 ]
 [0.28       0.86994366]
 [0.28       0.86353563]
 [0.28       0

In [27]:
max_wavelength = x['Wavelength'].max()
min_output = x['Out'].min()
max_output = x['Out'].max()
output_range = max_output - min_output

model = tf.keras.Sequential([
  layers.Dense(10, activation='relu', input_shape=(2,)),
  layers.Dropout(0.1),
  layers.Dense(10, activation='relu', input_shape=(2,)),  # Hidden layer
  layers.Dropout(0.1),
  layers.Dense(2)  # Output layer with 2 neurons
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(rs_X_train, rs_Y_train, epochs=2000, validation_data=(rs_X_val, rs_Y_val), verbose = 2)

Epoch 1/2000
35/35 - 1s - 34ms/step - loss: 3.8762 - val_loss: 4.5345
Epoch 2/2000
35/35 - 0s - 3ms/step - loss: 3.4065 - val_loss: 3.9211
Epoch 3/2000
35/35 - 0s - 2ms/step - loss: 2.9412 - val_loss: 3.3151
Epoch 4/2000
35/35 - 0s - 2ms/step - loss: 2.5166 - val_loss: 2.7612
Epoch 5/2000
35/35 - 0s - 2ms/step - loss: 2.2039 - val_loss: 2.3645
Epoch 6/2000
35/35 - 0s - 3ms/step - loss: 2.0119 - val_loss: 2.1409
Epoch 7/2000
35/35 - 0s - 2ms/step - loss: 1.9427 - val_loss: 2.0112
Epoch 8/2000
35/35 - 0s - 2ms/step - loss: 1.8609 - val_loss: 1.9343
Epoch 9/2000
35/35 - 0s - 2ms/step - loss: 1.8691 - val_loss: 1.8958
Epoch 10/2000
35/35 - 0s - 3ms/step - loss: 1.8462 - val_loss: 1.8736
Epoch 11/2000
35/35 - 0s - 2ms/step - loss: 1.8419 - val_loss: 1.8569
Epoch 12/2000
35/35 - 0s - 2ms/step - loss: 1.8045 - val_loss: 1.8450
Epoch 13/2000
35/35 - 0s - 2ms/step - loss: 1.8094 - val_loss: 1.8194
Epoch 14/2000
35/35 - 0s - 2ms/step - loss: 1.7869 - val_loss: 1.8159
Epoch 15/2000
35/35 - 0s - 2